# **Resumo da Análise Exploratória - Parte 01**
Esse notebook aborda uma breve exploração de dados para comprender pontos principais dos dados e a viabilidade para gerar um modelo de recomendação de produtos.

**Insights Gerados**
- 90% dos pedidos tem um único produto e 96% possuem um único produto ou mais de um produto igual.
- Como 96% da base consiste em pedidos com o mesmo produto, será possível utilizar o order_id como uma proxy para o produto, já que as bases da Olist não possuem a avalição feita por produto, mas apenas por pedido.
- Aproximadamente 97% dos usuários fizeram um único pedido.
- Numa escala de nota de 1 a 5: aproximadamente 12% dos usuários deram uma nota baixa para o produto (1 ou 2), 8% deram uma nota mediana (3) e 80% deram uma nota alta (4 ou 5).
- Mesmo com 97% dos usuários tendo um pedido, vamos no modelo testar tanto o valor absoluto da média de notas quanto os valores normalizado.

In [1]:
#Bibliotecas
import pandas as pd

In [2]:
#Bases de dados
olist_customers = pd.read_csv("olist_customers_dataset.csv", sep=",", 
                              usecols=['customer_id', 'customer_unique_id'])
olist_orders = pd.read_csv("olist_orders_dataset.csv", sep=",", 
                           usecols=['customer_id', 'order_id', 'order_status', 'order_purchase_timestamp'])
olist_order_reviews = pd.read_csv("olist_order_reviews_dataset.csv", sep=",", 
                                  usecols=['order_id', 'review_score'])
olist_order_items = pd.read_csv("olist_order_items_dataset.csv", sep=",", 
                                usecols=['order_id', 'product_id'])

---
# **Verificando a volumetria de pedidos e produtos de cada pedido**

O dataset **olist_order_items** contém a lista de pedidos e seus itens com diversas informações, mas para nós nesse momento vai ser importante apenas as colunas de **order_id** e **product_id** de cada pedido realizado.

Nessa seção do notebook, vamos verificar a volumetria de pedidos e entender alguns elementos principais, como: existem quantos itens por pedido? esses itens são iguais?

Numa exploração anterior nos datasets da Olist, foi percebido que essas bases tem avaliação de satisfação por pedido e não produto. A intenção agora é comprender essas carecterísicas para validar a possibilidade de utilizar o **order_id** com uma proxy de **product_id** para gerar o modelo de recomendação de produtos para os clientes desse dataset.

In [3]:
produtos = olist_order_items.groupby('order_id').agg(total=('product_id','count'),
                                          distinto=('product_id','nunique'))
produtos.reset_index(inplace=True)

In [4]:
produtos_df = pd.DataFrame({
    'volumetria_pedidos':produtos['total'].value_counts(),
    'volumetria_pedidos_perc':(produtos['total'].value_counts(normalize=True)*100).round(2),
})
produtos_df.head()

,volumetria_pedidos,volumetria_pedidos_perc
total,,
1,88863,90.06
2,7516,7.62
3,1322,1.34
4,505,0.51
5,204,0.21


In [5]:
produtos_df = pd.DataFrame({
    'volumetria_disintos':produtos['distinto'].value_counts(),
    'volumetria_disintos_perc':(produtos['distinto'].value_counts(normalize=True)*100).round(2),
})
produtos_df.head()

,volumetria_disintos,volumetria_disintos_perc
distinto,,
1,95430,96.72
2,2846,2.88
3,298,0.30
4,70,0.07
6,10,0.01


In [6]:
lista_de_pedidos_com_unico_produto = produtos[produtos['distinto'] == 1]['order_id']

- Aproximadamente 90% dos pedidos realizados que contém apenas um produto e 7% que contém 2 produtos.
- Aproximadamente 96% dos pedidos tem o mesmo produto enquanto 4% possuiam produtos diferentes.
- Como 96% da base consiste em pedidos com o mesmo produto, será possível utilizar o order_id como uma proxy para o produto, já que as bases da Olist não possuem a avalição feita por produto, mas apenas por pedido.
- Tendo essa informação, podemos gerar uma lista de order_id que pedidos que contém um item únicos ou vários itens iguais para utilizarmos no modelo de recomeção de outros produtos.

---
# **Verificando a volumetria de notas para cada pedido**

Nessa seção, vamos investigar os dados sobre outra ótica. Agora vamos verificar a viabilidade dos datasets sobre a quantidade de avaliações realizadas para esses pedidos da lista de order_id gerada na seção anterior. Assim podemos conferir se houve avaliações o suficiente para que seja confiável o uso dessa base para treinar o modelo de recomendação.

In [7]:
#Filtrando os order_id que estão dentro da lista de pedidos com único tipo de produto
df_produtos = olist_orders[olist_orders['order_id'].isin(lista_de_pedidos_com_unico_produto)]

#Comparando a qtd de linhas
df_produtos.shape[0], olist_orders.shape[0]

(95430, 99441)

Além de filtra o dataset **olist_orders** com a lista de order_ids da primeira seção, vamos também filtrar a base para pegarmos apenas os pedidos que tenham o **status como entregue**. Optei por essa forma para apenas utilizarmos apenas avaliações de produtos que foram devidamente entregues e consequentemente avaliados pelos os usuários.

In [8]:
df_produtos_entregues = df_produtos[df_produtos['order_status'] == 'delivered'][['customer_id', 'order_id']]

#Comparando a qtd de linhas
df_produtos_entregues.shape[0], df_produtos.shape[0], olist_orders.shape[0]

(93281, 95430, 99441)

Um fator importante sobre os datasets é que na base **olist_customers** existe dois IDs de usuários diferentes, um que único e outros que é usado para a conexão com as demais bases. Vamos fazer o cruzamento desse dataset com o nosso dataset atual para utilizarmos a coluna de **customer_unique_id** ao invés da coluna **customer_id**. Além de cruzar com o dataset **olist_order_reviws** para obtermos as avaliações feitas desses pedidos.

In [9]:
#Cruzamentos
df_produtos_avaliacoes = df_produtos_entregues.merge(olist_customers, on="customer_id", how="inner") \
                                              .merge(olist_order_reviews, on="order_id", how='left')
df_produtos_avaliacoes.drop(columns='customer_id', inplace=True)
df_produtos_avaliacoes.head()

,order_id,customer_unique_id,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,4.0
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,4.0
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,5.0
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,5.0
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,5.0


In [10]:
# Verificando se há valores vazios
df_produtos_avaliacoes.isna().value_counts()

order_id  customer_unique_id  review_score
False     False               False           93163
                              True              608
Name: count, dtype: int64

In [11]:
#Filtrando os pedidos que estão sem avaliações
df_produtos_avaliacoes.dropna(subset=['review_score'], inplace=True)

In [12]:
# Renomeando o nome das colunas
df_produtos_avaliacoes.rename(columns={
    'customer_unique_id':'user_id',
    'order_id':'product_id',
    'review_score':'rating'
}, inplace=True)

In [13]:
#Comparando a quantidade de usuários e comparação a quantidade de avalições feitas
avaliacoes_por_user_id = df_produtos_avaliacoes['user_id'].value_counts()
usuarios_por_qtd_avaliacoes = avaliacoes_por_user_id.value_counts().sort_index()
usuarios_por_qtd_avaliacoes_df = pd.DataFrame({
    'qtd_avaliacoes': usuarios_por_qtd_avaliacoes.index,
    'qtd_usuarios': usuarios_por_qtd_avaliacoes.values,
    'qtd_usuarios_perct':((usuarios_por_qtd_avaliacoes.values / usuarios_por_qtd_avaliacoes.values.sum()) * 100).round(2)
})
usuarios_por_qtd_avaliacoes_df

,qtd_avaliacoes,qtd_usuarios,qtd_usuarios_perct
0,1,87323,97.11
1,2,2154,2.40
2,3,310,0.34
3,4,101,0.11
4,5,22,0.02
5,6,11,0.01
6,7,1,0.00
7,15,1,0.00


In [22]:
usuarios = df_produtos_avaliacoes.groupby('user_id').agg(pedidos=('product_id','count'),
                                          avaliacoes=('rating','count'),
                                          avaliacoes_avg=('rating','mean'))
usuarios["avaliacoes_avg"] = usuarios["avaliacoes_avg"].round(0)
usuarios.reset_index(inplace=True)

In [15]:
usuarios_df = pd.DataFrame({
    'volumetria_usuarios':usuarios['pedidos'].value_counts(),
    'volumetria_usuarios_perc':(usuarios['pedidos'].value_counts(normalize=True)*100).round(2)
})
usuarios_df.head()

,volumetria_usuarios,volumetria_usuarios_perc
pedidos,,
1,87323,97.11
2,2154,2.40
3,310,0.34
4,101,0.11
5,22,0.02


In [24]:
usuarios_df = pd.DataFrame({
    'volumetria_usuarios':usuarios['avaliacoes_avg'].value_counts(),
    'volumetria_usuarios_perc':(usuarios['avaliacoes_avg'].value_counts(normalize=True)*100).round(2)
})
usuarios_df.sort_index()

,volumetria_usuarios,volumetria_usuarios_perc
avaliacoes_avg,,
1.0,8155,9.07
2.0,2706,3.01
3.0,7383,8.21
4.0,18300,20.35
5.0,53379,59.36


In [17]:
#Criando as notas normilizadas
df_produtos_avaliacoes['user_mean'] = df_produtos_avaliacoes.groupby('user_id')['rating'].transform('mean')
df_produtos_avaliacoes['user_reviews_count'] = df_produtos_avaliacoes.groupby('user_id')['rating'].transform('count')
df_produtos_avaliacoes['normalized_rating'] = df_produtos_avaliacoes['rating'] - df_produtos_avaliacoes['user_mean']

- Aproximadamente 97% dos usuários fizeram um único pedido.
- Numa escala de nota de 1 a 5: aproximadamente 12% dos usuários deram uma nota baixa para o produto (1 ou 2), 8% deram uma nota mediana (3) e 80% deram uma nota alta (4 ou 5).
- Mesmo com 97% dos usuários tendo um pedido, vamos no modelo testar tanto o valor absoluto da média de notas quanto os valores normalizado.